# Create figures using located tracks

## Libraries

In [ ]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'Helvetica'
plt.rcParams["font.size"] = 6

## Function

In [ ]:
def velocity(data):
    '''
    Calculate velocity from detected track
    '''
    res = np.sqrt(data)
    res = res.diff()/(1/40)
    res = res.mean()
    
    return res

## Analysis

In [ ]:
path = '../result/'

### Velocity distributions along channel width

In [ ]:
with open(path+'id553_on_off_1_track.txt', 'rb') as p:
    on_off_t = pickle.load(p)
    
with open(path+'id553_on_on_1_track.txt', 'rb') as p:
    on_on_t = pickle.load(p)

on_off_imsd = pd.read_csv(path+'id553_on_off_1_imsd.csv', index_col=0)
on_on_imsd = pd.read_csv(path+'id553_on_on_1_imsd.csv', index_col=0)

In [ ]:
on_off_t = on_off_t.drop_duplicates(subset='particle')
on_on_t = on_on_t.drop_duplicates(subset='particle')

In [ ]:
on_off_t

In [ ]:
on_on_t

In [ ]:
on_off_v = velocity(on_off_imsd)

In [ ]:
on_off_v.describe()

In [ ]:
on_off_t['Velocity'] = on_off_v.values
on_off_t['Hydrodynamic focusing'] = np.repeat('OFF', len(on_off_t))

In [ ]:
on_off_t

In [ ]:
on_on_v = velocity(on_on_imsd)

In [ ]:
on_on_t['Velocity'] = on_on_v.values
on_on_t['Hydrodynamic focusing'] = np.repeat('ON', len(on_on_t))

In [ ]:
on_on_t

In [ ]:
a_on_t = pd.concat([on_off_t, on_on_t])

In [ ]:
a_on_t['y'] = a_on_t['y']/1.124

In [ ]:
plt.figure(figsize=(1.41, 1.41))
sns.scatterplot(data=a_on_t, x='Velocity', y='y', hue='Hydrodynamic focusing', alpha=0.8, s=5)
plt.xlim(0, 800)
plt.xlabel(r'Velocity  [$\mu$$m$/$sec$]')
plt.ylabel(r'Position  [$\mu$$m$]')
plt.legend(title='Hydrodynamic\n     focusing', ncol=2, columnspacing=-0.2, markerscale=0.2, handletextpad=-0.5, loc='center left', borderaxespad=0.1, frameon=False)
plt.savefig(path+'velocity_distribution_along_width_9sec.pdf', bbox_inches='tight', pad_inches=0.05)

### Compare four conditions

In [ ]:
off_off_imsd = pd.read_csv(path+'id553_off_off_1_imsd.csv', index_col=0)
off_on_imsd = pd.read_csv(path+'id553_off_on_1_imsd.csv', index_col=0)
on_off_imsd = pd.read_csv(path+'id553_on_off_1_imsd.csv', index_col=0)
on_on_imsd = pd.read_csv(path+'id553_on_on_1_imsd.csv', index_col=0)

In [ ]:
res_off_off = velocity(off_off_imsd)
res_off_on = velocity(off_on_imsd)
res_on_off = velocity(on_off_imsd)
res_on_on = velocity(on_on_imsd)

In [ ]:
res = pd.DataFrame({'Velocity': pd.concat([res_off_off, res_off_on, res_on_off, res_on_on])})
res['Focusing'] = np.concatenate([np.repeat('OFF & OFF', len(res_off_off)), np.repeat('OFF & ON', len(res_off_on)), 
                                 np.repeat('ON & OFF', len(res_on_off)), np.repeat('ON & ON', len(res_on_on))])

In [ ]:
res

In [ ]:
plt.figure(figsize=(1.41, 1.41))
sns.violinplot(data=res, y='Velocity', x='Focusing', linewidth=0.3)
plt.ylim(0, 800)
plt.xlabel('Condition\n(Acousitic & Hydrodynamic)')
plt.ylabel(r'Velocity  [$\mu$$m$/$sec$]')
plt.xticks(rotation=45)
plt.savefig(path+'velocity_distribution_four_conditions_9sec.pdf', bbox_inches='tight', pad_inches=0.05)

In [ ]:
res_mean = res.groupby(['Focusing']).mean()
res_std = res.groupby(['Focusing']).std()

# calculate coefficient of variation
res_cv = res_std / res_mean

In [ ]:
res_mean

In [ ]:
res_std

In [ ]:
res_cv